In [1]:
import os


print(f"{os.getcwd()}")
print(f"{os.listdir()}")

/home/georger/WorkDocuments/Sxoli/Ptyxiaki/Ptyxiaki-Repository/Python_Scripts/Data_Analysis
['Powerfleet_API_Management.py', 'JSON_response.json', '__pycache__', 'data_analysis.ipynb', 'response_data_set.csv', 'Powerfleet_API_CredentialsI.txt']


Convert **JSON** to **CSV** so that **pandas** be able to use it


In [1]:
import json
import pandas as pd
from Powerfleet_API_Management import LivePowerfleetAPIManager as LivePowerfleetAPI
from colorama import Fore, Style
import sys  # For printing caught exceptions

if __name__ == "__main__":
    
    api_manager = LivePowerfleetAPI()  # Get data

    output_file = "response_data_set.csv"

    response_data = api_manager.retrieve_response()
    
    with open('JSON_response.json', 'w') as json_file:
        json.dump(response_data, json_file, indent=4)

    try:
        
        normalized_data = pd.json_normalize(response_data.values())    

        # Write the normalized DataFrame to a CSV file
        normalized_data.to_csv(output_file, index=False)

        print(Fore.GREEN + f"CSV file saved as {output_file}")
        print(Style.RESET_ALL)

    except ValueError as e:
        print(f"Error: The JSON file structure is not appropriate for conversion. {e}", file=sys.stderr)


/home/georger/WorkDocuments/Sxoli/Ptyxiaki/Ptyxiaki-Repository/Python_Scripts/Data_Analysis
os.getcwd()='/home/georger/WorkDocuments/Sxoli/Ptyxiaki/Ptyxiaki-Repository/Python_Scripts/Data_Analysis'
['Powerfleet_API_Management.py', 'Powerfleet_API_Credentials.txt', 'JSON_response.json', '__pycache__', 'data_analysis.ipynb', 'response_data_set.csv']
Headers: {'Content-Type': 'application/json', 'Authorization': '00595c0d-01d4-38a3-ba06-8852a0c6b2b8'}
Data: {'plate': '', 'cid': '946'}

Success: 200
{
    "11": {
        "date": "23/09/2024 12:01:14",
        "fuelLevel": "-1.0",
        "lng": "22.3707916",
        "vehicleId": "11",
        "velocity": "22.0",
        "ignition": "0",
        "lat": "37.5311933",
        "hashValue": "00441775020425039985"
    },
    "12": {
        "date": "23/09/2024 10:59:52",
        "fuelLevel": "-1.0",
        "lng": "22.3817983",
        "vehicleId": "12",
        "velocity": "29.0",
        "ignition": "0",
        "lat": "37.517725",
        "ha

Visualize data **dispersion**